In [14]:
import asyncio
from dotenv import load_dotenv
import aiohttp

load_dotenv()
import nest_asyncio

nest_asyncio.apply()
import os
from anthropic import AsyncAnthropic
from xml.sax.saxutils import escape
from suno import SongsGen
from concurrent.futures import ThreadPoolExecutor

In [15]:
import json
# Load json from data/data.json

with open('data/data.json') as f:
    data = json.load(f)
print(data['music_description'])

Playful and quirky instrumental background music with a light, whimsical tone.


In [16]:
async def async_generate_music(text):
    i = SongsGen(os.environ.get("SUNO_COOKIE"))
    print(i.get_limit_left())
    loop = asyncio.get_running_loop()

    result = None
    # Use a ThreadPoolExecutor to run synchronous functions in threads
    with ThreadPoolExecutor() as pool:
        result = await loop.run_in_executor(
            pool,
            lambda: i.get_songs(text, make_instrumental=True),
        )

    if not result:
        return None

    link = result["song_url"]
    print("Link: ", link)

    attempt = 0
    retry_delay = 5
    async with aiohttp.ClientSession() as session:
        while attempt < 5:
            async with session.get(link) as response:
                if response.status == 200:
                    data = await response.read()
                    # Check if data is not empty
                    if data:
                        return data
                    else:
                        print("No data received, retrying in 5 seconds...")
                else:
                    print(
                        f"Failed to fetch the song, status code: {response.status}, retrying in 5 seconds..."
                    )
            await asyncio.sleep(retry_delay)  # Async sleep for retry_delay seconds
            attempt += 1

    print("Failed to fetch the song after retries")
    return None

In [18]:
result = asyncio.run(async_generate_music(data['music_description']))

# Save result as mp3
with open('data/music.mp3', 'wb') as f:
    f.write(result)
    f.close()

190


Waiting for results...

.

.

.

...

.

.

.

Token expired, will sleep 30 seconds and try to download

Link:  https://audiopipe.suno.ai/?item_id=27fae954-bcd6-4544-8259-4346fd02e435
